# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770860415241                   -0.52    9.0    195ms
  2   -2.772146813225       -2.89       -1.33    1.0    119ms
  3   -2.772170140256       -4.63       -2.41    1.0    112ms
  4   -2.772170642648       -6.30       -3.10    1.0    115ms
  5   -2.772170722790       -7.10       -4.50    2.0    130ms
  6   -2.772170722959       -9.77       -4.78    1.0    133ms
  7   -2.772170723013      -10.27       -5.46    2.0    542ms
  8   -2.772170723015      -11.69       -6.13    1.0    116ms
  9   -2.772170723015      -12.83       -6.87    2.0    148ms
 10   -2.772170723015      -13.82       -7.15    1.0    123ms
 11   -2.772170723015   +  -14.12       -8.11    1.0    129ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770808794767                   -0.52    9.0    177ms
  2   -2.7

1.773557797493536

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem. This leads to a density-functional perturbation
theory problem, which is automatically set up and solved in the background.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770705933131                   -0.53    9.0    178ms
  2   -2.772050212547       -2.87       -1.31    1.0    120ms
  3   -2.772082647958       -4.49       -2.60    1.0    111ms
  4   -2.772083416085       -6.11       -4.14    2.0    129ms
  5   -2.772083417755       -8.78       -4.61    2.0    130ms
  6   -2.772083417809      -10.27       -5.81    1.0    551ms
  7   -2.772083417811      -11.72       -6.33    2.0    126ms
  8   -2.772083417811      -13.68       -7.27    1.0    111ms
  9   -2.772083417811   +  -14.51       -7.52    2.0    133ms
 10   -2.772083417811      -15.05       -7.85    1.0    119ms
 11   -2.772083417811      -14.35       -8.84    1.0    118ms

Polarizability via ForwardDiff:       1.772534966876748
Polarizability via finite difference: 1.773557797493536
